In [68]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-poster')
import warnings; warnings.simplefilter('ignore')

In [69]:
data = pd.read_csv('lyrics_titles_AutoPump.csv')

In [74]:
data.head()

,Unnamed: 0,title,lyrics
0,0,1 Choppa 2 Choppa*,"[Intro]\r\nAyy, ayy\r\n\r\n[Chorus]\r\nOne cho..."
1,0,30's,"[Verse 1: Lil Pump]\r\nI'ma hit a stain, I'ma ..."
2,0,70 Nigga,\r\n Lyrics for this song...
3,0,Aight,"[Intro: Lil Pump]\r\nYuh, ouu, ouu, ouu, ouu, ..."
4,0,At the Door,"[Intro]\r\nOoh, Big Head on the beat\r\nLil Pu..."


In [77]:
all_lyrics = open('all_lyrics.txt', 'w')
for row in data.itertuples():
    text = row.lyrics
    title = row.title
    if text.startswith('     '):
        continue
    all_lyrics.write(title + '\n')
    all_lyrics.write('********************\n')
    all_lyrics.write(text + '\n')
    all_lyrics.write('********************\n\n\n')
all_lyrics.close()

In [78]:
lines = []
all_lyrics = open('all_lyrics.txt', 'r')
for line in all_lyrics:
    if line.startswith(' '):
        line = line.lstrip()
    line = line.lower()
    if line.startswith('[chorus'):
        lines.append('[chorus] \n')
    elif line.startswith('[verse'):
        lines.append('[verse] \n')
    elif line.startswith('[hook'):
        lines.append('[hook] \n')
    elif line.startswith('[intro'):
        lines.append('[intro] \n')
    elif line.startswith('[outro'):
        lines.append('[outro] \n')
    elif line.startswith('[bridge'):
        lines.append('[bridge] \n')
    elif line.startswith('[interlude'):
        lines.append('[interlude] \n')
    elif line.startswith('lyrics for this'):
        lines.append('\n')
    else:
        lines.append(line)
all_lyrics.close()

In [79]:
all_lyrics_cleaned = open('all_lyrics_cleaned.txt', 'w')
for line in lines:
    all_lyrics_cleaned.write(line)
all_lyrics_cleaned.close() 

In [80]:
# Read in the entire file.
f = open('all_lyrics_cleaned.txt')
corpus0 = f.read()
f.close()

# Separate the punctuation from the words, so that words with
# punctuation do not get counted as distinct from words without
# punctuation.  Same for new line characters.

corpus0 = corpus0.replace(',', ' ,')
corpus0 = corpus0.replace('(', ' ( ')
corpus0 = corpus0.replace(')', ' ) ')
#corpus0 = corpus0.replace('[', ' [ ')
#corpus0 = corpus0.replace(']', ' ] ')
corpus0 = corpus0.replace('.', ' . ')
corpus0 = corpus0.replace(';', ' ; ')
corpus0 = corpus0.replace(':', ' : ')
corpus0 = corpus0.replace('!', ' ! ')
corpus0 = corpus0.replace('?', ' ? ')
corpus0 = corpus0.replace('********************', ' ******************** ')
#corpus0 = corpus0.replace('*', ' * ')
corpus0 = corpus0.replace("’", '\'')
corpus0 = corpus0.replace("\'\'", ' " ')
corpus0 = corpus0.replace('"', ' " ')
corpus0 = corpus0.replace('\r\n', ' \r\n ')

# Separate the dashes from any words they're attached to.
corpus0 = corpus0.replace('-', ' - ')
corpus0 = corpus0.replace('\n', ' \n ')

# Convert the text to lower case.
corpus0 = corpus0.lower()

# Split the words by spaces; only take the first 500000 words.
# This number was chosen based on memory limits and training-time
# limitations.
corpus1 = corpus0.split(' ')

while (corpus1.count('') > 0): 
    corpus1.remove('')
    
print('Length of corpus is now ', len(corpus1))

Length of corpus is now  46584


In [81]:
# Preprocessing is done.  Now get the unique words, and encode them.
words = sorted(list(set(corpus1)))
num_words = len(words)
encoding = {w: i for i, w in enumerate(words)}
decoding = {i: w for i, w in enumerate(words)}

print('We have', num_words, 'different words.')
corpus = corpus1

We have 2500 different words.


In [83]:
# Chop up the data into x and y, slice into roughly num_chars
# overlapping 'sentences' of length sentence_length.  Encode the
# characters.
sentence_length = 50
x_data = []
y_data = []
for i in range(0, len(corpus) - sentence_length):
    sentence = corpus[i: i + sentence_length]
    next_word = corpus[i + sentence_length]
    x_data.append([encoding[word] for word in sentence])
    y_data.append(encoding[next_word])

# good word: phronesis
num_sentences = len(x_data)
print('We have', len(x_data), 'sentences.')

# Create the variables to hold the data as it will be used.
x = np.zeros((num_sentences, sentence_length, num_words), dtype = np.bool)
y = np.zeros((num_sentences, num_words), dtype = np.bool)

# Populate the sentences.
print('Encoding data.')
for i, sentence in enumerate(x_data):
    for t, encoded_word in enumerate(sentence):
        x[i, t, encoded_word] = 1
    y[i, y_data[i]] = 1

We have 46534 sentences.
Encoding data.


In [84]:
# The processing of the data takes a fair amount of time.  Save
# the data so we don't have to do this again.  We do this in a
# numpy file since the data is large and the shelve can't handle
# it.

print('Saving processed data.')
np.save('x.npy', x)
np.save('y.npy', y)

Saving processed data.


In [85]:
import keras.models as km
import keras.layers as kl

In [86]:
print('Building network.')
model = km.Sequential()
model.add(kl.LSTM(128, input_shape = (sentence_length, num_words)))
model.add(kl.Dense(num_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'sgd',
              metrics = ['accuracy'])

Building network.


In [ ]:
# Fit!  Begin elevator music...
print('Beginning fit.')
fit = model.fit(x, y, epochs = 200, batch_size = 256)

# Save the model so that we can use it as a starting point.
model.save('model_50.h5')

Beginning fit.
Epoch 1/200
46534/46534 [==============================] - 104s 2ms/step - loss: 5.5129 - acc: 0.1120
Epoch 2/200
46534/46534 [==============================] - 104s 2ms/step - loss: 5.4754 - acc: 0.1120
Epoch 3/200
46534/46534 [==============================] - 104s 2ms/step - loss: 5.4460 - acc: 0.1120
Epoch 4/200
46534/46534 [==============================] - 104s 2ms/step - loss: 5.4223 - acc: 0.1120
Epoch 5/200
46534/46534 [==============================] - 105s 2ms/step - loss: 5.4028 - acc: 0.1120
Epoch 6/200
46534/46534 [==============================] - 105s 2ms/step - loss: 5.3865 - acc: 0.1120
Epoch 7/200
14848/46534 [========>.....................] - ETA: 1:11 - loss: 5.3930 - acc: 0.1136

### This piece of code will load the saved model from the file and generate a song from the initial seed

In [61]:
import numpy as np

import keras.models as km
import shelve
import random

modelfile = 'model_50.h5'

#######################################################################

# Get the model.
print('Reading model file.')
model = km.load_model(modelfile)

# Get the meta-data.

sentence_length = 50
num_words = 2500


# Randomly choose 50 words from the dictionary of words as our
# starting sentence.
seed = []
for i in range(sentence_length):
    seed.append(decoding[random.randint(0, num_words - 1)])


# Encode the seed sentence.
x = np.zeros((1, sentence_length, num_words), dtype = np.bool)
for i, w in enumerate(seed):
    x[0, i, encoding[w]] = 1

text = ''

# Run the seed sentence through the model.  Add the output to the
# generated text.  Take the output and append it to the seed sentence
# and remove the first word from the seed sentence.  Then repeat until
# you've generated as many words as you like.
for i in range(150):

    # Get the most-probably next word.
    pred = np.argmax(model.predict(x, verbose = 0))

    # Add it to the generated text.
    text += decoding[pred] + ' '

    # Encode the next word.
    next_word = np.zeros((1, 1, num_words), dtype = np.bool)
    next_word[0, 0, pred] = 1

    # Concatenate the next word to the seed sentence, but leave off
    # the first element so that the length stays the same.
    x = np.concatenate((x[:, 1:, :], next_word), axis = 1)

    
# Print out the generated text.
print("Generated lyrics: \n")
print(text)

Reading model file.
Generated lyrics: 

on 
 came the , wok your whitney , cry , ooh , i do it 
 yeah , bitch , i ain't , these shit , ooh your stop ) 
 yeah , niggas in you we from like a coupe ( damn ) 
 she around this dick on make her is i i'm ) 
 i sell at the money , boy got a new and yeah 
 dropped bitch i got i new shit 
 jump i fuckin' in your bitch , go go jump 
 
 [ intro ] 
 lil pump lil pump , lil pump , lil pump , i don't up a bitch ( esskeetit ) 
 rather up go lean ) 
 i i just left 'em ( don't ) 
 to to xan at go , and i think off 
 
 [ intro ] 
 yeah pump , yeah 
